In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Required Libraries

In [2]:
!pip install -Uq transformers
!pip install -Uq evaluate
!pip install -Uq SentencePiece
!pip install -Uq sentence-transformers
!pip install rouge-score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-sc

## Load Libraries

In [55]:
import pandas as pd
import torch
import ast
from sentence_transformers import SentenceTransformer
from sentence_transformers import util


# import numpy as np
# import matplotlib.pyplot as plt
# import json
# from tqdm import tqdm

# import torch.nn as nn
# from torch.optim import Adam
# from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.nn.functional import cosine_similarity
# import torch.nn.functional as F

# import tensorflow as tf

# # import spacy
# # import string

# from sklearn.model_selection import train_test_split

import transformers
# from transformers import pipeline
import evaluate  # Bleu
from transformers import Trainer, TrainingArguments

import nltk
nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
# @title Load Configuraiton
import hbqaconfig, Utilfunc
kaggle=False
conf = hbqaconfig.setEnv('colab')

predict_now=False
embed_now=True
compute_metrics_now=True

In [5]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

DEVICE

device(type='cpu')

# Select & Load Embedding Model

In [6]:
embmodelname, embmodelshort, embmodelname1 = Utilfunc.select_embmodel(3)
# embmodel = SentenceTransformer(embmodelname)

multi-qa-mpnet-base-dot-v1 	 mpnet 	 _multi-qa-mpnet-base-dot-v1


# Load New Chunks and QA Dataset in Dataframe

In [7]:
df_chunk = pd.read_excel(conf['RAAGS_FOLDER'] + '10.1-HBQA-for-RAAGS.xlsx', sheet_name="Chunk")
df_qa = pd.read_excel(conf['RAAGS_FOLDER'] + '10.1-HBQA-for-RAAGS.xlsx', sheet_name="QA")

In [11]:
df_chunk.shape, df_qa.shape

((50, 2), (163, 4))

In [8]:
df_chunk.head(3)

,Chunk_Id,Chunk
0,1001,"This work is a tree, of which the chapter of c..."
1,1002,"This work is a tree, of which the chapter of c..."
2,1003,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...


In [9]:
df_qa.head(3)

,Ques_Id,Chunk_Id,Question,Ref_Answer
0,2001,1001,"Who recited the Mahabharata to the Devas, Pitr...","Narada recited the Mahabharata to the Devas, D..."
1,2002,1001,Who are compared to the fruit and flowers of t...,"The two sons of Madri, the twin Aswins, are co..."
2,2003,1001,How did the citizens react when the young prin...,The distinguished class of citizens shouted fo...


# Create New Chunk's and DA Dataset's Vector

## Chunk vector

In [13]:
# When some function is decorated with this function. It will show the time to execute that function.
# we will use this calculate the time for embedding.
def get_time(func):
    import time
    def wrapper():
        start_time = time.time()

        result = func()

        end_time = time.time()
        time_spent=end_time-start_time

        print(f"Time Taken to Process {time_spent}")
        return result

    return wrapper

In [ ]:
# Convert Chunk colum values to List.
@get_time
def get_chunk_vector():
  Chunk_Sentences = df_chunk.Chunk.tolist()
  Chunk_Embeddings = embmodel.encode(Chunk_Sentences)
  return Chunk_Embeddings

Chunk_Embeddings = get_chunk_vector()

# Convert These embedding into torch tensor and load into available DEVICE
ChunkVectors= torch.tensor(Chunk_Embeddings, dtype=torch.float).to(DEVICE)

# Flatten the tensors into 1D arrays/list
# Create ChunkVector in dataset.'
df_chunk['ChunkVector'+embmodelname1 ] = ChunkVectors.tolist()



Time Taken to Process 90.84815168380737


In [ ]:
print(df_chunk.shape)
df_chunk.head(3)

(50, 3)


,Chunk_Id,Chunk,ChunkVector_multi-qa-mpnet-base-dot-v1
0,1001,"This work is a tree, of which the chapter of c...","[0.23210011422634125, -0.17947782576084137, -0..."
1,1002,"This work is a tree, of which the chapter of c...","[0.23210011422634125, -0.17947782576084137, -0..."
2,1003,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,"[-0.0005072467029094696, -0.08607526123523712,..."


In [ ]:
# Save the Vector on harddisk.
filenm = "10.2-HBQA_RAAGS_chunk_Vector" + embmodelname1 +".csv"
df_chunk.to_csv(conf['RAAGS_FOLDER'] + filenm, index=None)

## QA Vector

In [ ]:
# Convert Chunk colum values to List.
@get_time
def get_qa_vector():
  Question_Sentences = df_qa.Question.tolist()
  Ref_Answer_Sentences = df_qa.Ref_Answer.tolist()
  Question_Embeddings = embmodel.encode(Question_Sentences)
  Ref_Answer_Embeddings = embmodel.encode(Ref_Answer_Sentences)
  return Question_Embeddings, Ref_Answer_Embeddings

Question_Embeddings, Ref_Answer_Embeddings = get_qa_vector()

Time Taken to Process 40.692105293273926


In [ ]:
# Convert These embedding into torch tensor and load into available DEVICE
QuestionVectors= torch.tensor(Question_Embeddings, dtype=torch.float).to(DEVICE)
Ref_AnswerVectors= torch.tensor(Ref_Answer_Embeddings, dtype=torch.float).to(DEVICE)

# Flatten the tensors into 1D arrays/list
# Create ChunkVector in dataset.'
df_qa['QuesVector'+embmodelname1 ] = QuestionVectors.tolist()
df_qa['AnsVector'+embmodelname1 ] = Ref_AnswerVectors.tolist()

print(embmodelname)

df_qa.head(3)

multi-qa-mpnet-base-dot-v1


,Ques_Id,Chunk_Id,Question,Ref_Answer,QuestionVector_multi-qa-mpnet-base-dot-v1,Ref_AnswerVector_multi-qa-mpnet-base-dot-v1
0,2001,1001,"Who recited the Mahabharata to the Devas, Pitr...","Narada recited the Mahabharata to the Devas, D...","[-0.1249421238899231, -0.025616921484470367, -...","[0.033066216856241226, -0.24593223631381989, -..."
1,2002,1001,"Narada recited the Mahabharata to the Devas, D...","The two sons of Madri, the twin Aswins, are co...","[0.03306629881262779, -0.24593225121498108, -0...","[-0.15711747109889984, -0.3908252716064453, -0..."
2,2003,1001,How did the citizens react when the young prin...,The distinguished class of citizens shouted fo...,"[0.18756987154483795, -0.16670864820480347, -0...","[0.01948237046599388, -0.008955039083957672, -..."


In [ ]:
# Save the Vector on harddisk.
filenm = "10.3-HBQA_RAAGS_QA_Vector" + embmodelname1 +".csv"
df_qa.to_csv(conf['RAAGS_FOLDER'] + filenm, index=None)

# Load New Chunks and QA Vector Dataset from Harddisk

In [38]:
filenm = "10.2-HBQA_RAAGS_chunk_Vector" + embmodelname1 +".csv"
df_chunk1 = pd.read_csv(conf['RAAGS_FOLDER'] + filenm)

filenm = "10.3-HBQA_RAAGS_QA_Vector" + embmodelname1 +".csv"
df_qa1 = pd.read_csv(conf['RAAGS_FOLDER'] + filenm)

# Load Chunks and Question
(QAS System is trained on this)

In [11]:
print(df_qa1.shape)
df_qa1.head(2)

(163, 6)


,Ques_Id,Chunk_Id,Question,Ref_Answer,QuesVector_multi-qa-mpnet-base-dot-v1,AnsVector_multi-qa-mpnet-base-dot-v1
0,2001,1001,"Who recited the Mahabharata to the Devas, Pitr...","Narada recited the Mahabharata to the Devas, D...","[-0.1249421238899231, -0.025616921484470367, -...","[0.033066216856241226, -0.24593223631381989, -..."
1,2002,1001,Who are compared to the fruit and flowers of t...,"The two sons of Madri, the twin Aswins, are co...","[0.03306629881262779, -0.24593225121498108, -0...","[-0.15711747109889984, -0.3908252716064453, -0..."


In [34]:
filenm= "07.2-HBQA_QA_Vector" + embmodelname1 +".csv"
print(filenm)
df_qa2 = pd.read_csv(conf['QAGS_FOLDER'] + filenm,\
                      usecols= ['Ques_Id', 'Chunk_Id', 'Question', 'Ref_Answer', \
                                'QuesVector' + embmodelname1, 'AnsVector' + embmodelname1])
# df_qa2 = df_qa2.set_index(df_qa2.Ques_Id)
print(df_qa2.shape)
df_qa2.head(2)

07.2-HBQA_QA_Vector_multi-qa-mpnet-base-dot-v1.csv
(1102, 6)


,Ques_Id,Chunk_Id,Question,Ref_Answer,QuesVector_multi-qa-mpnet-base-dot-v1,AnsVector_multi-qa-mpnet-base-dot-v1
0,0,389,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,"[-0.4276844561100006, 0.07686357200145721, -0....","[-0.316677063703537, 0.04548855870962143, -0.2..."
1,1,390,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"[-0.04931388795375824, -0.15976008772850037, -...","[-0.013333199545741081, 0.05390685796737671, -..."


In [13]:
print(df_chunk1.shape)
df_chunk1.head(2)

(50, 3)


,Chunk_Id,Chunk,ChunkVector_multi-qa-mpnet-base-dot-v1
0,1001,"This work is a tree, of which the chapter of c...","[0.23210011422634125, -0.17947782576084137, -0..."
1,1002,"This work is a tree, of which the chapter of c...","[0.23210011422634125, -0.17947782576084137, -0..."


In [33]:
filenm= '04.3-HBQA_Chunk_Vector'+embmodelname1+'.csv'
print(filenm)
df_chunk2 = pd.read_csv(conf['PE_FOLDER'] + filenm, usecols=['Chunk_Id','Chunk','ChunkVector'+embmodelname1])
print(df_chunk2.shape)
df_chunk2.head(2)

04.3-HBQA_Chunk_Vector_multi-qa-mpnet-base-dot-v1.csv
(334, 3)


,Chunk_Id,Chunk,ChunkVector_multi-qa-mpnet-base-dot-v1
0,389,Even this is eternal morality. They that perfo...,"[-0.044648099690675735, -0.0822770893573761, -..."
1,390,"Section III\n""Vaisampayana said, 'Yudhishthira...","[0.2307928055524826, -0.18484173715114594, -0...."


# Merge Chunks and QA Vector Datasets

In [39]:
# Concat Chunk and Reindex
df_chunk = pd.concat([df_chunk1, df_chunk2])
# df_chunk = df_chunk2.copy()

df_chunk.index = range(df_chunk.shape[0])
print(df_chunk.shape)
df_chunk.head(3)

(384, 3)


,Chunk_Id,Chunk,ChunkVector_multi-qa-mpnet-base-dot-v1
0,1001,"This work is a tree, of which the chapter of c...","[0.23210011422634125, -0.17947782576084137, -0..."
1,1002,"This work is a tree, of which the chapter of c...","[0.23210011422634125, -0.17947782576084137, -0..."
2,1003,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,"[-0.0005072467029094696, -0.08607526123523712,..."


In [40]:
df_chunk.tail(3)

,Chunk_Id,Chunk,ChunkVector_multi-qa-mpnet-base-dot-v1
381,961,"Section LXXXI\n""Sahadeva said, 'What hath been...","[0.15935538709163666, -0.3883747458457947, -0...."
382,974,"Thus consoled by Krishna, Kunti, afflicted wit...","[0.2113616168498993, -0.1391368806362152, -0.1..."
383,989,"Section CI\n""Narada said, 'This region belonge...","[0.08386620879173279, 0.02076539397239685, -0...."


In [41]:
# Concat Question and Reindex
df_qa = pd.concat([df_qa1, df_qa2])
# df_qa = df_qa2.copy()

df_qa.index = range(df_qa.shape[0])
print(df_qa.shape)
df_qa.head(3)

(1265, 6)


,Ques_Id,Chunk_Id,Question,Ref_Answer,QuesVector_multi-qa-mpnet-base-dot-v1,AnsVector_multi-qa-mpnet-base-dot-v1
0,2001,1001,"Who recited the Mahabharata to the Devas, Pitr...","Narada recited the Mahabharata to the Devas, D...","[-0.1249421238899231, -0.025616921484470367, -...","[0.033066216856241226, -0.24593223631381989, -..."
1,2002,1001,Who are compared to the fruit and flowers of t...,"The two sons of Madri, the twin Aswins, are co...","[0.03306629881262779, -0.24593225121498108, -0...","[-0.15711747109889984, -0.3908252716064453, -0..."
2,2003,1001,How did the citizens react when the young prin...,The distinguished class of citizens shouted fo...,"[0.18756987154483795, -0.16670864820480347, -0...","[0.01948237046599388, -0.008955039083957672, -..."


In [42]:
df_qa.tail(3)

,Ques_Id,Chunk_Id,Question,Ref_Answer,QuesVector_multi-qa-mpnet-base-dot-v1,AnsVector_multi-qa-mpnet-base-dot-v1
1262,1099,961,What is Sahadeva's desire and the reason behin...,Sahadeva desires to provoke a war with the Kau...,"[0.1654556393623352, -0.40023961663246155, -0....","[0.02510783076286316, -0.6113841533660889, -0...."
1263,1100,974,How did Kunti respond to Krishna's counsel reg...,Kunti responded by placing her full trust in K...,"[0.29927170276641846, 0.0765724703669548, -0.2...","[0.31003451347351074, -0.05984937772154808, -0..."
1264,1101,989,Can you provide the names of some prominent ch...,Narada listed several prominent chiefs of the ...,"[0.07110704481601715, -0.06087541580200195, -0...","[0.12445423007011414, -0.29634472727775574, -0..."


# Predict Chunk for Question

In [19]:
# Load all chunk_vectors into memotry
chunk_id = df_chunk['Chunk_Id'].to_list()

chunk_vector = [ast.literal_eval(df_chunk['ChunkVector'+embmodelname1][i]) for i in range(df_chunk.shape[0])]
# chunk_vector = [df_chunk['ChunkVector'+embmodelname1][i] for i in range(df_chunk.shape[0])]
chunk_vector = torch.tensor(chunk_vector)

In [20]:
# Search Chunk based on Each Question. Get 5 chunk for each question
df_chunk_pred = pd.DataFrame(columns = ['Ques_Id','Chunk_Id','Pred_Chunk_Id'])

for i in df_qa.index:
  question_embedding = ast.literal_eval( df_qa.loc[ i,'QuesVector' + embmodelname1])
  question_embedding = torch.tensor(question_embedding)

  # print (type(question_embedding), question_embedding)
  top_k=5
  hits = util.semantic_search(question_embedding, chunk_vector, top_k=top_k)


  predictedId= [item['corpus_id'] for item in hits[0]] # Hit contains index (corpus_id), location of chunk_vector
  predicted_Docid = [df_chunk.loc[i, "Chunk_Id"] for i in predictedId] # We need to get chunk_id corresponding to chunk_index
  # print (predicted_Docid)
  df_chunk_pred.loc[i]= (df_qa.loc[i, 'Ques_Id'], df_qa.loc[i,'Chunk_Id'], predicted_Docid) # save predictions
  print('Predicting: ',i)
  # break

Predicting:  0
Predicting:  1
Predicting:  2
Predicting:  3
Predicting:  4
Predicting:  5
Predicting:  6
Predicting:  7
Predicting:  8
Predicting:  9
Predicting:  10
Predicting:  11
Predicting:  12
Predicting:  13
Predicting:  14
Predicting:  15
Predicting:  16
Predicting:  17
Predicting:  18
Predicting:  19
Predicting:  20
Predicting:  21
Predicting:  22
Predicting:  23
Predicting:  24
Predicting:  25
Predicting:  26
Predicting:  27
Predicting:  28
Predicting:  29
Predicting:  30
Predicting:  31
Predicting:  32
Predicting:  33
Predicting:  34
Predicting:  35
Predicting:  36
Predicting:  37
Predicting:  38
Predicting:  39
Predicting:  40
Predicting:  41
Predicting:  42
Predicting:  43
Predicting:  44
Predicting:  45
Predicting:  46
Predicting:  47
Predicting:  48
Predicting:  49
Predicting:  50
Predicting:  51
Predicting:  52
Predicting:  53
Predicting:  54
Predicting:  55
Predicting:  56
Predicting:  57
Predicting:  58
Predicting:  59
Predicting:  60
Predicting:  61
Predicting:  62
Pr

In [21]:
tmpfilenm = "10.4-Predicted-DocumentId-for-Ques"+embmodelname1+".csv"
df_chunk_pred.to_csv( conf['RAAGS_FOLDER'] + tmpfilenm,index=None)

In [22]:
# Mean Reciprocal Rank (MRR) is a metric commonly used to evaluate the performance of ranking-based retrieval systems,
# such as search engines. MRR assesses how well a retrieval system ranks relevant items by considering the position of the
# first relevant item in the ranked list. It's calculated as the average of the reciprocal ranks across a set of queries.

def getReciprocalRank(row):
  K=1 # Number of relevent document
  try:
    i = row['Pred_Chunk_Id'].index( row['Chunk_Id']) #ast.literal_eval( row['Pred_Chunk_Id']).index(row['Chunk_Id'] )
  except:
    i=-1

  if i==-1:
    RR=0
  else:
    RR = round(1/(i + 1),2)

#   print (i)
  return RR

df_chunk_pred['RR'] = df_chunk_pred.apply(getReciprocalRank, axis=1)

In [23]:
df_chunk_pred.head(10)

,Ques_Id,Chunk_Id,Pred_Chunk_Id,RR
0,2001,1001,"[76, 1003, 693, 690, 593]",0.00
1,2002,1001,"[76, 704, 1004, 602, 703]",0.00
2,2003,1001,"[1006, 212, 811, 395, 707]",0.00
3,2004,1001,"[235, 345, 477, 453, 717]",0.00
4,2005,1002,"[1001, 1002, 76, 1005, 499]",0.50
5,2006,1002,"[1001, 1002, 1010, 596, 1005]",0.50
6,2007,1002,"[557, 452, 578, 593, 693]",0.00
7,2008,1002,"[1006, 212, 712, 702, 155]",0.00
8,2009,1003,"[605, 76, 702, 1003, 583]",0.25
9,2010,1003,"[76, 1004, 1003, 1010, 645]",0.33


In [24]:
df_chunk_pred['RR'].mean()

0.5420474308300396

In [25]:
df_chunk_pred.sort_values('RR',ascending=False)

,Ques_Id,Chunk_Id,Pred_Chunk_Id,RR
1264,1101,989,"[989, 1022, 69, 469, 1034]",1.0
397,234,480,"[480, 470, 476, 460, 451]",1.0
760,597,614,"[614, 453, 805, 696, 706]",1.0
392,229,478,"[478, 669, 696, 433, 477]",1.0
393,230,478,"[478, 470, 477, 480, 735]",1.0
...,...,...,...,...
637,474,556,"[395, 235, 783, 393, 398]",0.0
1,2002,1001,"[76, 704, 1004, 602, 703]",0.0
631,468,555,"[502, 515, 520, 556, 553]",0.0
629,466,555,"[515, 556, 520, 502, 583]",0.0


# Select & Load Prediction Model

## Select Prediction Model

In [7]:
predmodels_list = {"0" :"T5:t5", "1" : "Flan-T5:flant5", "2":"GPT2:gpt2","3":"DistilBERT:distilbert",
                   "4":"RoBERTa:roberta","5":"Llama2:llama2", "6":"BERTSquad:bert",
                   "7":"LongFormer:longformer", "8":"DistilBERT512:distilbert512"}
# Longformer doesn't work with latest Transformer
predmodel_name = predmodels_list['0'].split(':')[1]
print(predmodel_name)

t5


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
conf['MODEL_FOLDER'] = '/content/drive/MyDrive/HBQA/t5small-30epoch'

# Load the corresponding tokenizer
tokenizer = T5Tokenizer.from_pretrained( conf['MODEL_FOLDER'])

# Load the pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained(conf['MODEL_FOLDER'] )
model.to(DEVICE)

# Predict Answers from t5-30ep-finetuned model


In [33]:
# @title Function: Predict Answer
def predict_answer(context, question, tokenizer, model):
#     Q_LEN=1500

#     if predmodel_name=="t5":
    inputs = tokenizer(question, context, max_length= Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=100)

    predicted_answer = tokenizer.decode(outputs.flatten(), skip_special_tokens=True)


    if len(predicted_answer)<3:
       predicted_answer="xxx"
    elif predicted_answer[0:5]=='[CLS]' or  predicted_answer[0:5]=='[SEP]' or predicted_answer[0:3]=='<s>' :
        predicted_answer="xxx"
    return predicted_answer

In [34]:
# @title Function: Predict Multipel Answer for Question and Corresponding Predicted Chunk
from IPython.display import clear_output
clear_output()

from IPython.display import display
from IPython.display import HTML

# Load file into dataframe for prediction.
#Get 5 answer from top 5 document for each question
tmpfilenm = "10.4-Predicted-DocumentId-for-Ques"+embmodelname1+".csv"
df = pd.read_csv( conf['RAAGS_FOLDER'] + tmpfilenm)

if predict_now:
  df_pred_answer = pd.DataFrame(columns=['Ques_Id','Question','Ref_Answer','Pred_FinalAns','Pred_Answer1','Pred_Answer2', 'Pred_Answer3', 'Pred_Answer4', 'Pred_Answer5'])

def predict_all_answers(df_pred, sample=True, verbose=False):
    import random
    if sample:
        ques_idx=random.sample( set(df.index),10)
    else:
        ques_idx = df.index

    total_items = len(ques_idx)

    # progress_bar = tqdm(total=total_items, position=0, leave=True)
    # progress_bar = tqdm(total=total_items, position=0, leave=True, dynamic_ncols=True)
    # progress_html = HTML("""<progress value="0" max="{0}" style="width: 100%"></progress>""".format(total_items))


    for i  in ques_idx:

        #   progress_bar.update(j)
        #   progress_html.value = """<progress value="{0}" max="{1}" style="width: 100%"></progress>""".format(i, total_items)
        #   progress_bar = tqdm(total=total_items, position=0, leave=True)

        ids = df.loc[i,"Pred_Chunk_Id"]
        ques_id = df.loc[i,"Ques_Id"]
        print ("Predicting for Question Id: ",ques_id)

        cond     = df_qa["Ques_Id"] == ques_id
        ques     = df_qa.loc[cond]["Question"].values[0]
        ref_ans  = df_qa.loc[cond]['Ref_Answer'].values[0]

        # print(ques)

        if len(ids)>3:
            ids = ast.literal_eval(ids)
        else:
            print("Sorry, No document")
            break

        if verbose:
            print('Question  :', ques)
            print("Ref Answer:", ref_ans)

        ans=[]
        for id in ids:
            # print(ques_id, id)
            cond  = df_chunk["Chunk_Id"]==id
            chunk = df_chunk.loc[cond]["Chunk"].values[0]

            # print(chunk[:20])
            pred_ans = predict_answer(chunk, ques, tokenizer, model)
            ans.append( pred_ans )

            if verbose:
                print("Pred Ans  :", pred_ans)


        # final prediction with all the joint answers.
        pred_finalans =  predict_answer(" ".join(ans), ques, tokenizer, model)

        if verbose:
            print("Final Pred Ans  :", pred_finalans)
            print('--------')

        df_pred_answer.loc[i] = ques_id, ques, ref_ans, pred_finalans, ans[0], ans[1], ans[2], ans[3], ans[4]

        # print(f"Prediting Answer {i}/{df.shape[0]}")

  # progress_bar.close()

In [35]:
Q_LEN =  1500 #256   # Question Length
# T_LEN =  500 #32  # Target Length
# BATCH_SIZE = 4

In [ ]:
# @title Start Predicting Answer & then save prediction
tmpfilenm = "10.5-Predicted_Multiple_Answer"+embmodelname1+".csv"

if predict_now:
  predict_all_answers(df_pred_answer, sample=False, verbose=True)

if predict_now:
  print('Saving Predictions')
  df_pred_answer.to_csv( conf['RAAGS_FOLDER'] + tmpfilenm,index=None)

In [43]:
df_pred_answer

,Ques_Id,Question,Ref_Answer,Pred_FinalAns,Pred_Answer1,Pred_Answer2,Pred_Answer3,Pred_Answer4,Pred_Answer5
0,2001,"Who recited the Mahabharata to the Devas, Pitr...","Narada recited the Mahabharata to the Devas, D...","The Brahmanas, King Yudhishthira, recited the ...","Krishna-Dwaipayana, the Brahmanas, recited the...",The Mahadivya was recited by Krishna-Dwaipayan...,The Mahabharata was recited by King Yudhishthi...,The Mahabharata recited the Mahabharata to the...,"The Mahabharata was recited by the Devas, Pitr..."
1,2002,Who are compared to the fruit and flowers of t...,"The two sons of Madri, the twin Aswins, are co...",The fruit and flowers of the vast tree formed ...,The fruits and flowers of the vast tree formed...,The fruit and flowers of the vast tree formed ...,The fruits and flowers of the vast tree formed...,The fruits and flowers of the vast tree formed...,The fruit and flowers of the vast tree formed ...
2,2003,How did the citizens react when the young prin...,The distinguished class of citizens shouted fo...,"When the young princes, the Pandavas, were int...","When the young princes, the Pandavas, were int...","The citizens, including the princes Dhrishtady...","When the young princes, the Pandavas, were int...","When the young princes, the Pandavas, were int...",The citizens were greatly affected by grief an...
3,2004,Who obtained the virgin Krishna at the swayamv...,Arjuna obtained the virgin Krishna at the sway...,The virgin Krishna at the swayamvara was obtai...,The virgin Krishna obtained by Dhrishtadyumna ...,"The virtuous Hari, thus addressed by Yudhishth...",The virgin Krishna obtained by King Rituparna ...,The virgin Krishna at the swayamvara was obtai...,The virgin Krishna obtained by Jayadratha at t...
4,2005,What is the significance of the book named Mau...,The book called Mausala is described as an epi...,The book named Mausala is a vast tree formed o...,The book named Mausala is a vast tree formed o...,The book named Mausala is a vast tree formed o...,Mausala is a sacred and excellent book written...,The book named Mausala is considered an epitom...,Mausala is a book that is described as a sacre...
...,...,...,...,...,...,...,...,...,...
1260,1097,"In this passage, how does Dhritarashtra descri...",Dhritarashtra describes Lord Krishna (Vasudeva...,Dhritarashtra describes Lord Krishna (Vasudeva...,Dhritarashtra describes Lord Krishna (Vasudeva...,Dhritarashtra describes Lord Krishna (Vasudeva...,Dhritarashtra describes Lord Krishna (Vasudeva...,Dhritarashtra describes Lord Krishna (Vasudeva...,Dhritarashtra describes Lord Krishna (Vasudeva...
1261,1098,What does Yudhishthira request of Krishna in t...,Yudhishthira requests Krishna to go to the Kur...,Yudhishthira asks Krishna to bring peace to th...,Yudhishthira asks Krishna to make a peace to a...,Yudhishthira requests Krishna to return to his...,Yudhishthira demands Krishna to protect Mahend...,Yudhishthira asks Krishna to worship Lord Brah...,Yudhishthira demands Krishna to restrain his h...
1262,1099,What is Sahadeva's desire and the reason behin...,Sahadeva desires to provoke a war with the Kau...,Sahadeva's desire is to bring Nala to Ayodhya ...,Sahadeva's desire is to seek the help of King ...,"Sahadeva desires to bring Nala to Ayodhya, str...",Sahadeva's desire is to calmly subdue his risi...,Sahadeva's desire is to abandon virtue and avo...,Sahadeva desires to go to Vasudeva's car and w...
1263,1100,How did Kunti respond to Krishna's counsel reg...,Kunti responded by placing her full trust in K...,Kunti expressed his desire to intervene in the...,Kunti responded to Krishna's counsel by saying...,Kunti responded to Krishna's counsel by dispel...,"Kunti, speaking to Krishna's counsel, expresse...","Kunti, upon hearing Krishna's counsel's counse...",Kunti expressed his concern about the course o...


# Load Embedding Model

In [37]:
#Select Model Function

# https://www.sbert.net/docs/pretrained_models.html

#250MB, multi-qa-distilbert-cos-v1',  Max Sequence Length:	512, Dimensions:768, Normalized Embeddings:	true
#80MB, all-MiniLM-L6-v2, Max Sequence Length:	256, Dimensions:	384, Normalized Embeddings:	true
#290MB, all-distilroberta-v1, Max Sequence Length:	512, Dimensions:	768, Normalized Embeddings:	true
#420MB, all-mpnet-base-v2, Max Sequence Length:	384, Dimensions:	768, Normalized Embeddings:	true
#1.36GB, all-roberta-large-v1, Max Sequence Length:	256, Dimensions: 1024, Normalized Embeddings:	true

def select_embmodel(num):
    emb_modelshortlist = ['distilbert','minilm','distilroberta','mpnet','roberta']

    emb_modellist = ['multi-qa-distilbert-cos-v1',
                'all-MiniLM-L6-v2',
                'all-distilroberta-v1',
                'multi-qa-mpnet-base-dot-v1',
                'all-roberta-large-v1']

    embmodelname = emb_modellist[num]
    embmodelshort = emb_modelshortlist[num]
    embmodelname1 = "_" + embmodelname

    print (embmodelname,'\t',embmodelshort,'\t', embmodelname1)
    return embmodelname, embmodelshort, embmodelname1

In [38]:
embmodelname, embmodelshort, embmodelname1 = select_embmodel(3)

multi-qa-mpnet-base-dot-v1 	 mpnet 	 _multi-qa-mpnet-base-dot-v1


In [9]:
embed_now=True

In [10]:
if embed_now:
  embmodel = SentenceTransformer(embmodelname)

In [11]:
tmpfilenm = "10.5-Predicted_Multiple_Answer"+embmodelname1+".csv"
df_pred_answer1 =  pd.read_csv(conf['RAAGS_FOLDER'] + tmpfilenm)

# df_pred_answer1.set_index('Ques_Id',inplace=True)

In [12]:
df_pred_answer1.head(3)

,Ques_Id,Question,Ref_Answer,Pred_FinalAns,Pred_Answer1,Pred_Answer2,Pred_Answer3,Pred_Answer4,Pred_Answer5
0,2001,"Who recited the Mahabharata to the Devas, Pitr...","Narada recited the Mahabharata to the Devas, D...","The Brahmanas, King Yudhishthira, recited the ...","Krishna-Dwaipayana, the Brahmanas, recited the...",The Mahadivya was recited by Krishna-Dwaipayan...,The Mahabharata was recited by King Yudhishthi...,The Mahabharata recited the Mahabharata to the...,"The Mahabharata was recited by the Devas, Pitr..."
1,2002,Who are compared to the fruit and flowers of t...,"The two sons of Madri, the twin Aswins, are co...",The fruit and flowers of the vast tree formed ...,The fruits and flowers of the vast tree formed...,The fruit and flowers of the vast tree formed ...,The fruits and flowers of the vast tree formed...,The fruits and flowers of the vast tree formed...,The fruit and flowers of the vast tree formed ...
2,2003,How did the citizens react when the young prin...,The distinguished class of citizens shouted fo...,"When the young princes, the Pandavas, were int...","When the young princes, the Pandavas, were int...","The citizens, including the princes Dhrishtady...","When the young princes, the Pandavas, were int...","When the young princes, the Pandavas, were int...",The citizens were greatly affected by grief an...


In [13]:
def create_ans_vector():

    Ans_Sentences = df_pred_answer1.Pred_FinalAns.tolist()
    Ans_Embeddings = embmodel.encode(Ans_Sentences)

    print ("Answer Embedding: ", Ans_Embeddings.shape)
    # return (Ques_Embeddings, Ans_Embeddings)

    return (Ans_Embeddings)

In [14]:
# @title Answer not found these questions
df_pred_answer1['Pred_FinalAns'] = df_pred_answer1.Pred_FinalAns.fillna("xxx")
df_pred_answer1.loc[df_pred_answer1['Pred_FinalAns'].str.contains(r'\?\?\?'), 'Pred_FinalAns'] = "xxx"

# Model couldn't fine any any answer for this
df_pred_answer1.loc[df_pred_answer1['Pred_FinalAns'].str.contains(r"xxx")].shape

(0, 9)

In [15]:
if embed_now:
  Ans_Embeddings = create_ans_vector()
  df_pred_answer1['Pred_Answer_Vector'] = torch.tensor(Ans_Embeddings, dtype=torch.float).tolist()

Answer Embedding:  (1265, 768)


In [16]:

predicted_ansvec_file = '10.6-' + predmodel_name +'Predicted_AnsVec-Finetuned.csv'
if embed_now:
  df_pred_answer1.to_csv(conf['RAAGS_FOLDER'] + predicted_ansvec_file)

# df_pred = pd.read_csv(datapath + '09.11-' + predmodel_name +'Predicted_AnsVec-Zeroshot.csv')
print(df_pred_answer1.shape)
df_pred_answer1.head(2)

(1265, 10)


,Ques_Id,Question,Ref_Answer,Pred_FinalAns,Pred_Answer1,Pred_Answer2,Pred_Answer3,Pred_Answer4,Pred_Answer5,Pred_Answer_Vector
0,2001,"Who recited the Mahabharata to the Devas, Pitr...","Narada recited the Mahabharata to the Devas, D...","The Brahmanas, King Yudhishthira, recited the ...","Krishna-Dwaipayana, the Brahmanas, recited the...",The Mahadivya was recited by Krishna-Dwaipayan...,The Mahabharata was recited by King Yudhishthi...,The Mahabharata recited the Mahabharata to the...,"The Mahabharata was recited by the Devas, Pitr...","[0.009074064902961254, -0.25097066164016724, -..."
1,2002,Who are compared to the fruit and flowers of t...,"The two sons of Madri, the twin Aswins, are co...",The fruit and flowers of the vast tree formed ...,The fruits and flowers of the vast tree formed...,The fruit and flowers of the vast tree formed ...,The fruits and flowers of the vast tree formed...,The fruits and flowers of the vast tree formed...,The fruit and flowers of the vast tree formed ...,"[-0.21235719323158264, -0.6044186353683472, 0...."


# Calculate Metrics

In [17]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics import precision_score, recall_score
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer
import numpy as np

smoother = SmoothingFunction()
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

def get_nlp_metrics(ques_id, ref_ans, pred_ans):

  # Calculate ROUGE-1 and ROUGE-L scores
  scores = scorer.score(ref_ans, pred_ans)

  # Access individual ROUGE scores
  rouge1_precision = scores['rouge1'].precision
  rouge1_recall = scores['rouge1'].recall
  rouge1_f1 = scores['rouge1'].fmeasure

  rougel_precision = scores['rougeL'].precision
  rougel_recall = scores['rougeL'].recall
  rougel_f1 = scores['rougeL'].fmeasure

#   rouge_1_precision, rouge_1_recall, rouge_1_f1, rouge_l_precision, rouge_l_recall, rouge_l_f1,

  ref_tokens = word_tokenize(ref_ans)
  pred_tokens = word_tokenize(pred_ans)

  # Calculate BLEU score for a single sentence
  bleu_score = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoother.method2)

  # Calculate BLEU score for multiple sentences
  # corpus_bleu_score = corpus_bleu([[ref_tokens]], [pred_tokens],  smoothing_function=smoother.method2)

  # print(set(ref_tokens).intersection(set(pred_tokens)))
  tp = len(set(ref_tokens).intersection(set(pred_tokens)))
  precision = tp / len(pred_tokens)
  recall =  tp / len(ref_tokens)

  return ques_id, bleu_score, rouge1_precision, rouge1_recall, rouge1_f1, rougel_precision, rougel_recall, rougel_f1, precision, recall

In [21]:
# from datasets import load_metric
if compute_metrics_now:
  df_metrics = pd.DataFrame(columns = ['Ques_Id','BLEU1', 'ROUGE1_P', 'ROUGE1_R', 'ROUGE1_F1',
                              'ROUGEL_P', 'ROUGEL_R', 'ROUGEL_F1', 'Precision', 'Recall'])
  N= df_pred_answer1.shape[0]

  for i in range(N):
    # bleu_score1 = calculate_score( df1.iloc[i]['ref_answer'],df1.iloc[i]['pred_answer'])
    ques_id = df_pred_answer1.loc[i, "Ques_Id"]
    ref_ans  = df_pred_answer1.loc[i,'Ref_Answer']
    pred_ans = df_pred_answer1.loc[i,'Pred_FinalAns']
    results = get_nlp_metrics(ques_id, ref_ans, pred_ans)
    df_metrics.loc[i] = results

In [ ]:
# !rm /kaggle/working/09.21-t5Predicted_AnsVec-Zeroshot.csv

In [26]:
# @title Write Metrics in Score file
metricsfile = '10.7-' + predmodel_name +'Predicted_Ans_Score_Finetuned.csv'
print(metricsfile)

if compute_metrics_now:
  df_metrics.to_csv(conf['REPORT_FOLDER'] + metricsfile, index=False)

# df_metrics = pd.read_csv(datapath + metricsfile)
print(df_metrics.shape)
df_metrics.head(3)

10.7-t5Predicted_Ans_Score_Finetuned.csv
(1265, 10)


,Ques_Id,BLEU1,ROUGE1_P,ROUGE1_R,ROUGE1_F1,ROUGEL_P,ROUGEL_R,ROUGEL_F1,Precision,Recall
0,2001.0,0.266677,0.538462,0.538462,0.538462,0.50,0.500000,0.500000,0.305556,0.343750
1,2002.0,0.257341,0.340000,0.708333,0.459459,0.34,0.708333,0.459459,0.288462,0.576923
2,2003.0,0.068306,0.340909,0.357143,0.348837,0.25,0.261905,0.255814,0.200000,0.227273


In [28]:
# @title Read metrics from Score file

metricsfile = '10.7-' + predmodel_name +'Predicted_Ans_Score_Finetuned.csv'
df_metrics = pd.read_csv(conf['RAAGS_FOLDER'] + metricsfile)
print(df_metrics.shape)
df_metrics.head(3)

(1265, 10)


,Ques_Id,BLEU1,ROUGE1_P,ROUGE1_R,ROUGE1_F1,ROUGEL_P,ROUGEL_R,ROUGEL_F1,Precision,Recall
0,2001.0,0.266677,0.538462,0.538462,0.538462,0.50,0.500000,0.500000,0.305556,0.343750
1,2002.0,0.257341,0.340000,0.708333,0.459459,0.34,0.708333,0.459459,0.288462,0.576923
2,2003.0,0.068306,0.340909,0.357143,0.348837,0.25,0.261905,0.255814,0.200000,0.227273


# Calculate Cosine between Predicted and Reference Answer

In [46]:
# @title Load Ref_Answers in dataframe
filenm = "10.3-HBQA_RAAGS_QA_Vector" + embmodelname1 +".csv"
df_qa1 = pd.read_csv(conf['RAAGS_FOLDER'] + filenm)

filenm= "07.2-HBQA_QA_Vector" + embmodelname1 +".csv"
df_qa2 = pd.read_csv(conf['QAGS_FOLDER'] + filenm,\
                      usecols= ['Ques_Id', 'Chunk_Id', 'Question', 'Ref_Answer', \
                                'QuesVector' + embmodelname1, 'AnsVector' + embmodelname1])

df_ref_ans = pd.concat([df_qa1, df_qa2])

In [50]:
print(df_ref_ans.shape)
df_ref_ans.head(2)

(1265, 6)


,Ques_Id,Chunk_Id,Question,Ref_Answer,QuesVector_multi-qa-mpnet-base-dot-v1,AnsVector_multi-qa-mpnet-base-dot-v1
0,2001,1001,"Who recited the Mahabharata to the Devas, Pitr...","Narada recited the Mahabharata to the Devas, D...","[-0.1249421238899231, -0.025616921484470367, -...","[0.033066216856241226, -0.24593223631381989, -..."
1,2002,1001,Who are compared to the fruit and flowers of t...,"The two sons of Madri, the twin Aswins, are co...","[0.03306629881262779, -0.24593225121498108, -0...","[-0.15711747109889984, -0.3908252716064453, -0..."


In [48]:
# @title Join Predicted Answer and Reference Dataframes
df_ref_pred= df_pred_answer1.merge(df_ref_ans, on = "Ques_Id", how="inner")[['Ques_Id','Pred_Answer_Vector','AnsVector_multi-qa-mpnet-base-dot-v1']]

In [52]:
print(df_ref_pred.shape)
df_ref_pred.head(2)

(1265, 3)


,Ques_Id,Pred_Answer_Vector,AnsVector_multi-qa-mpnet-base-dot-v1
0,2001,"[0.009074064902961254, -0.25097066164016724, -...","[0.033066216856241226, -0.24593223631381989, -..."
1,2002,"[-0.21235719323158264, -0.6044186353683472, 0....","[-0.15711747109889984, -0.3908252716064453, -0..."


In [53]:
def calculate_cosine(row):


    if type(row['Pred_Answer_Vector'])!=type(list()):
      predAns_vector_values = ast.literal_eval(row['Pred_Answer_Vector'])
    else:
      predAns_vector_values = row['Pred_Answer_Vector']

    if type(row['AnsVector' + embmodelname1])!=type(list()):
      refAns_vector_values = ast.literal_eval(row['AnsVector' + embmodelname1])
    else:
      refAns_vector_values = row['AnsVector' + embmodelname1]


    # Convert the string values to floats
    predAns_vector_values = [float(value) for value in predAns_vector_values]
    refAns_vector_values = [float(value) for value in refAns_vector_values]

    # Convert the lists to PyTorch tensors
    predAns_vector_values = torch.tensor(predAns_vector_values).reshape(1, -1)
    refAns_vector_values = torch.tensor(refAns_vector_values).reshape(1, -1)

    similarity = cosine_similarity(predAns_vector_values, refAns_vector_values)
    return similarity.item()

In [56]:
df_ref_pred['Cosine'] = df_ref_pred.apply(calculate_cosine, axis=1)

In [57]:
df_metrics = df_metrics.merge(df_ref_pred, on="Ques_Id", how="left")[['Ques_Id', 'BLEU1',
                                                                      'ROUGE1_P', 'ROUGE1_R', 'ROUGE1_F1',
                                                                      'ROUGEL_P', 'ROUGEL_R', 'ROUGEL_F1',
                                                                      'Precision', 'Recall','Cosine']]

# Save File Metrics & Show Summary

In [62]:
metricsfile = '10.7-' + predmodel_name +'Predicted_Ans_Score_Finetuned.csv'
print(metricsfile)
df_metrics.to_csv(conf['RAAGS_FOLDER']+ metricsfile, index=False)
df_metrics.head(2)

10.7-t5Predicted_Ans_Score_Finetuned.csv


,Ques_Id,BLEU1,ROUGE1_P,ROUGE1_R,ROUGE1_F1,ROUGEL_P,ROUGEL_R,ROUGEL_F1,Precision,Recall,Cosine
0,2001.0,0.266677,0.538462,0.538462,0.538462,0.50,0.500000,0.500000,0.305556,0.343750,0.793567
1,2002.0,0.257341,0.340000,0.708333,0.459459,0.34,0.708333,0.459459,0.288462,0.576923,0.642552


In [60]:
all_metrics = pd.DataFrame()
all_metrics[predmodel_name] = df_metrics.mean()[1:]

In [61]:
# all_metrics.rename(columns = {'t5':'t5-small-1epoch'}, inplace=True)
all_metrics

,t5
BLEU1,0.140149
ROUGE1_P,0.411307
ROUGE1_R,0.393121
ROUGE1_F1,0.383841
ROUGEL_P,0.323270
ROUGEL_R,0.312898
ROUGEL_F1,0.303076
Precision,0.316105
Recall,0.303884
Cosine,0.763268


In [ ]:
df_metrics.mean()

Ques_Id      550.500000
BLEU1          0.196152
ROUGE1_P       0.512886
ROUGE1_R       0.455007
ROUGE1_F1      0.465596
ROUGEL_P       0.408665
ROUGEL_R       0.366577
ROUGEL_F1      0.372750
Precision      0.393385
Recall         0.350023
Cosine         0.826508
dtype: float64